In [ ]:
import numpy as np
from scipy.stats.stats import spearmanr, pearsonr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import pandas as pd
from matplotlib import style
style.use("ggplot")
from sklearn import svm, preprocessing
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
import pickle
from scipy.signal import find_peaks_cwt

doc_id = []
year = []
citation_count = []
rounds = []
accepted = []
with open('accepted_papers_all_info_final.txt') as infile:
    for line in infile:
        a = line.split()
        doc_id.append(a[0])
        year.append(a[1])
        rounds.append(a[2])
        citation_count.append(a[3])
        accepted.append(1)
        
with open('rejected_papers_all_info_final.txt') as infile:
    for line in infile:
        a = line.split()
        doc_id.append(a[0])
        year.append(a[1])
        rounds.append(a[2])
        citation_count.append(a[3])
        accepted.append(0)
        
d = {'doc_id' : doc_id,
     'citations' : citation_count,
     'rounds':rounds,
     'year' : year,
     'accepted' : accepted}
document_details = pd.DataFrame(d)

doc =[]
auth = []
with open('DocIdAuthorId.txt') as infile:
    for line in infile:
        a = line.split()
        doc.append(a[0])
        auth.append(a[1])
        
d = {'doc_id' : doc,
     'author_id' : auth}
author_doc = pd.DataFrame(d)

document_details = pd.merge(document_details , author_doc , on='doc_id' , how = 'inner')
document_details.year = document_details.year.astype(int)
document_details.citations = document_details.citations.astype(int)
document_details.rounds = document_details.rounds.astype(int)
document_details.doc_id = document_details.doc_id.astype(int)
document_details.author_id = document_details.author_id.astype(int)
document_details = document_details.drop_duplicates()
y = document_details.author_id.unique()
author_profile = dict()
for i in y:
    author_profile[i] = []
    a = document_details[document_details['author_id']==i]
    b = min(a.year)
    for j in list(range(b, 2015)):
        c = a[a['year']==j]
        d = c[c['accepted']==1]
        e = c.shape[0]
        if e==0:
            author_profile[i].append(0)
        else:
            author_profile[i].append(d.shape[0]/float(e))

high_accept = []
low_accept = []
medium_accept = []

for author in author_profile:
    d = author_profile[author]
    e = 0.0
    if len(d)==0:
        continue
    for i in d:
        if i>.7:
            e +=1
    if e/len(d)>.7:
        high_accept.append(author)
        continue
    e = 0.0
    for i in d:
        if i<.4:
            e +=1
    if e/len(d)>.8:
        low_accept.append(author)
        continue
    medium_accept.append(author)

out = open("./categories_new.txt", "w")
out1 = open("./high_acceptance.txt", "w")
out2 = open("./low_acceptance.txt", "w")
out3 = open("./medium_acceptance.txt", "w")

for author in high_accept:
    out.write(str(author) + ",1\n")
    out1.write(str(author) + ":"+ " ".join(str(x) for x in author_profile[author]) + "\n")
for author in low_accept:
    out.write(str(author) + ",2\n")
    out2.write(str(author) + ":"+ " ".join(str(x) for x in author_profile[author]) + "\n")
for author in medium_accept:
    out.write(str(author) + ",3\n")
    out3.write(str(author) + ":"+ " ".join(str(x) for x in author_profile[author]) + "\n")

out.close()
out1.close()
out2.close()
out3.close()

# Topic Diversity

version = []
keyword = []
with open('versionkeywordsTable.txt') as infile:
    for line in infile:
        a = line.split()
        version.append(a[0])
        keyword.append(a[1])
        
d = {
    'version_id':version,
    'keyword':keyword
}
topic = pd.DataFrame(d)
topic.keyword = topic.keyword.astype(int)
topic.version_id = topic.version_id.astype(int)
version = []
auth = []
with open('versionCod_DocCod.txt') as infile:
    for line in infile:
        a = line.split()
        version.append(a[0])
        auth.append(a[1])
        
d = {
    'version_id':version,
    'doc_id':auth
}
df = pd.DataFrame(d)
df.doc_id = df.doc_id.astype(int)
df.version_id = df.version_id.astype(int)
df = pd.merge(topic,df)
df = df[['doc_id','keyword']]
df = df.drop_duplicates()

#High Accept
topichigh = [[]]
for i in high_accept:
    l = document_details[document_details['author_id']==i]['doc_id'].unique()
    c = []
    for j in l:
        c.extend(df[df['doc_id']==j]['keyword'].unique())
    topichigh.append(c)
    
#Low Accept
topiclow = []
for i in low_accept:
    l = document_details[document_details['author_id']==i]['doc_id'].unique()
    c = []
    for j in l:
        c.extend(df[df['doc_id']==j]['keyword'].unique())
    topiclow.append(c)
    
#Medium Accept
topicmed = []
for i in medium_accept:
    l = document_details[document_details['author_id']==i]['doc_id'].unique()
    c = []
    for j in l:
        c.extend(df[df['doc_id']==j]['keyword'].unique())
    topicmed.append(c)
    
    
# Topic Diversity
numhigh = []
for i in high_accept:
    numhigh.append(document_details[document_details['author_id']==i].shape[0])

d = {
    'author_id':high_accept,
    'topic':topichigh[1:],
    'number':numhigh
}
df1 = pd.DataFrame(d)

l = []
for index,row in df1.iterrows():
    l.append(len(set(row['topic']))/float(row['number']))
    
df1['Diversity'] = l

numlow = []
for i in low_accept:
    numlow.append(document_details[document_details['author_id']==i].shape[0])

d = {
    'author_id':low_accept,
    'topic':topiclow,
    'number':numlow
}
df2 = pd.DataFrame(d)

l = []
for index,row in df2.iterrows():
    l.append(len(set(row['topic']))/float(row['number']))
    
df2['Diversity'] = l

nummed = []
for i in medium_accept:
    nummed.append(document_details[document_details['author_id']==i].shape[0])

d = {
    'author_id':medium_accept,
    'topic':topicmed,
    'number':nummed
}
df3 = pd.DataFrame(d)

l = []
for index,row in df3.iterrows():
    l.append(len(set(row['topic']))/float(row['number']))
    
df3['Diversity'] = l

df1 = df1[df1['number']>3]
df2 = df2[df2['number']>3]
df3 = df3[df3['number']>3]
# High
df1['Diversity'].mean()
# Low
df2['Diversity'].mean()
# Medium
df3['Diversity'].mean()


# Final Dataset containing all features
df = pd.read_csv('Final_dataset.csv')
high_accept = []
with open('high_acceptance.txt') as infile:
    for line in infile:
        a = line.split(':')
        high_accept.append(int(a[0]))
low_accept = []
with open('low_acceptance.txt') as infile:
    for line in infile:
        a = line.split(':')
        low_accept.append(int(a[0]))
med_accept = []
with open('medium_acceptance.txt') as infile:
    for line in infile:
        a = line.split(':')
        med_accept.append(int(a[0]))
        
# Citations

l1 = []
for i in high_accept:
    a = df[df['author_id']==i]['citations']
    if len(a)==0 : continue
    l1.extend(a)
print sum(l1)/len(l1)


l2 = []
for i in low_accept:
    a = df[df['author_id']==i]['citations']
    if len(a)==0 : continue
    l2.extend(a)
print sum(l2)/len(l2)

l3 = []
for i in med_accept:
    a = df[df['author_id']==i]['citations']
    if len(a)==0 : continue
    l3.extend(a)
print sum(l3)/len(l3)

print np.std(l1)
print np.std(l2)
print np.std(l3)

# Closeness Centrality
l4 = []
for i in high_accept:
    a = df[df['author_id']==i]['closeness_centrality']
    if len(a)==0 : continue
    l4.extend(a)
np.mean(l4)

l5 = []
for i in low_accept:
    a = df[df['author_id']==i]['closeness_centrality']
    if len(a)==0 : continue
    l5.extend(a)
np.mean(l5)

l6 = []
for i in medium_accept:
    a = df[df['author_id']==i]['closeness_centrality']
    if len(a)==0 : continue
    l6.extend(a)
np.mean(l6)

# Sentiment
rl = pd.read_csv('Sentiment.csv')
l4 = []
for i in high_accept:
    a = rl[rl['author_id']==i]['Len']
    if len(a)==0 : continue
    l4.extend(a)
np.mean(l4)

l4 = []
for i in med_accept:
    a = rl[rl['author_id']==i]['Len']
    if len(a)==0 : continue
    l4.extend(a)
np.mean(l4)

l4 = []
for i in low_accept:
    a = rl[rl['author_id']==i]['Len']
    if len(a)==0 : continue
    l4.extend(a)
np.mean(l4)

# Number of Rounds ( Accepted and Rejected)

l7a = []
l7r = []
for i in high_accept:
    a = df[df['author_id']==i]
    b = a[a['accepted']==1]['rounds']
    c = a[a['accepted']==0]['rounds']
    if len(b)==0 :
        pass
    else:
        l7a.extend(b)
    if len(c)==0 :
        pass
    else:
        l7r.extend(c)
print np.mean(l7a)
print np.mean(l7r)

l8a = []
l8r = []
for i in low_accept:
    a = df[df['author_id']==i]
    b = a[a['accepted']==1]['rounds']
    c = a[a['accepted']==0]['rounds']
    if len(b)==0 :
        pass
    else:
        l8a.extend(b)
    if len(c)==0 :
        pass
    else:
        l8r.extend(c)
print np.mean(l8a)
print np.mean(l8r)

l9a = []
l9r = []
for i in med_accept:
    a = df[df['author_id']==i]
    b = a[a['accepted']==1]['rounds']
    c = a[a['accepted']==0]['rounds']
    if len(b)==0 :
        pass
    else:
        l9a.extend(b)
    if len(c)==0 :
        pass
    else:
        l9r.extend(c)
print np.mean(l9a)
print np.mean(l9r)

# Experience

l10 = []
for i in high_accept:
    a = df[df['author_id']==i]
    if a.shape[0]==0:
        continue
    l10.append(a.shape[0])
np.mean(l10)

l11 = []
for i in low_accept:
    a = df[df['author_id']==i]
    if a.shape[0]==0:
        continue
    l11.append(a.shape[0])
np.mean(l11)

l12 = []
for i in med_accept:
    a = df[df['author_id']==i]
    if a.shape[0]==0:
        continue
    l12.append(a.shape[0])
np.mean(l12)

from collections import Counter
s = pd.Series(l10)
s.value_counts()[:10].plot('bar')
plt.xlabel('Experience')
plt.ylabel('Frequency')
plt.title('High Accept')
plt.show()

s = pd.Series(l11)
s.value_counts()[:10].plot('bar')
plt.xlabel('Experience')
plt.ylabel('Frequency')
plt.title('Low Accept')
plt.show()

s = pd.Series(l12)
s.value_counts()[:10].plot('bar')
plt.xlabel('Experience')
plt.ylabel('Frequency')
plt.title('Medium Accept')
plt.show()

